In [1]:
import warnings
warnings.filterwarnings("ignore")


import requests

import pandas as pd
import numpy as np
from scipy import stats
from math import sqrt
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.tsa.api import Holt

from datetime import datetime

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit 

#from wrangle import get_crypto_price

import wrangle as w
import modeling as m

In [ ]:
btc = w.get_crypto_price('btc', '2018-01-01', '2022-12-12')

btc = w.clean_data(btc)

In [ ]:
# human splitting by year
train = btc[:'2020']
validate = btc['2021']
test = btc['2022']

col = 'btc_volume'

# Modeling ig

In [ ]:
# Create the empty dataframe
eval_df = pd.DataFrame(columns=['model_type', 'target_var', 'rmse'])

# Initialize volume, yhat_df, and period for modeling
volume = 0 #train['btc_volume'][-1:][0]

yhat_df = pd.DataFrame({'btc_volume': [volume]}, 
                       index = validate.index)

period = 0

## Last Observed

In [ ]:
volume, yhat_df = get_btc_last_observed(train, validate, volume, yhat_df)

In [ ]:
eval_df = append_eval_df(model_type = 'last_observed_value', 
                             target_var = col)
eval_df

In [ ]:
plot_and_eval(col)

## Simple Average

In [ ]:
volume, yhat_df = get_btc_simple_average(train, validate, volume, yhat_df)

In [ ]:
eval_df = append_eval_df(model_type='simple_average', 
                             target_var = col)
eval_df

In [ ]:
plot_and_eval(col)

# Moving Average

In [ ]:
volume, yhat_df = get_btc_30d_average(train, validate, volume, yhat_df)

In [ ]:
eval_df = append_eval_df(model_type='30d_moving_average', 
                             target_var = col)
eval_df

In [ ]:
plot_and_eval(col)

In [2]:
periods = [1,7,14,21,28,120]

In [ ]:
for p in periods:
        volume = round(train['btc_volume'].rolling(p).mean().iloc[-1], 2)


        yhat_df = pd.DataFrame({'btc_volume': [volume]}, 
                           index = validate.index)

        model_type = str(p) + 'd moving average'
        eval_df = append_eval_df(model_type = model_type, 
                                 target_var = col)

In [ ]:
eval_df

# Previous Cycle

In [ ]:
yhat_df = get_btc_previous_cycle(train, validate)

In [ ]:
plot_and_eval(target_var = col)

In [ ]:
eval_df = append_eval_df(model_type = 'previous year', target_var = col)

In [ ]:
eval_df

# Model on Test

In [ ]:
period = 21
volume = round(train['btc_volume'].rolling(period).mean().iloc[-1], 2)


yhat_df = pd.DataFrame({'btc_volume': [volume]}, 
                       index = test.index)

In [ ]:
plot_and_eval_test(col)